In [ ]:
import pandas as pd
label = pd.read_csv("/home/iatell/projects/meta-learning/data/seq_line_labels.csv")
label["seq_len"] = label["endIndex"] - label["startIndex"]
label

In [2]:

import pandas as pd
df = pd.read_csv("/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv")
df

,timestamp,open,high,low,close,volume,upper_shadow,body,lower_shadow,Candle_Color,upper_body_ratio,lower_body_ratio,upper_lower_body_ratio
0,2018-01-01,13707.91,13818.55,12750.00,13380.00,8607.15640,0.076003,-0.225254,0.432772,1,0.337410,1.921259,0.175619
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.16540,0.540071,0.874627,0.332912,2,0.617487,0.380633,1.622262
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.48210,0.263644,0.155931,0.366880,2,1.690776,2.352839,0.718611
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,25224.41500,0.150006,0.095280,0.681423,2,1.574377,5.000000,0.220136
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.35200,0.144690,1.274181,0.308056,2,0.113556,0.241768,0.469688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,2022-05-19,28715.33,30545.18,28691.38,30319.23,67877.36415,0.109006,0.773779,0.011554,2,0.140875,0.014932,5.000000
1600,2022-05-20,30319.22,30777.33,28730.00,29201.01,60517.25325,0.221063,-0.539597,0.227288,1,0.409682,0.421218,0.972612
1601,2022-05-21,29201.01,29656.18,28947.28,29445.06,20987.13124,0.103235,0.119338,0.124071,2,0.865069,1.039664,0.832066
1602,2022-05-22,29445.07,30487.99,29255.11,30293.94,36158.98748,0.095648,0.418411,0.093632,2,0.228598,0.223780,1.021531


# model


## FNNCNN

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] # REMOVED redundant transposes

        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU

        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    def configure_optimizers(self): 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode="min",
    #         factor=0.2,   # Reduce LR by 80%
    #         patience=5,   # After 5 epochs of no val_loss improvement
    #         verbose=True
    #     )
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # Important!
    #         },
        # }


## CNNlSTM

In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] # REMOVED redundant transposes

        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU
        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    # Inside your CNNLSTM_MDN class
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
    #     # Define the scheduler
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode='min',      # We want to minimize the validation loss
    #         factor=0.5,      # Reduce LR by 50% (1.0 -> 0.2)
    #         patience=10,      # Wait 5 validation epochs with no improvement before reducing
    #         verbose=True
    #     )
        
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # The metric to watch
    #         },
    #     }
    
    def configure_optimizers(self): 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode="min",
    #         factor=0.2,   # Reduce LR by 80%
    #         patience=5,   # After 5 epochs of no val_loss improvement
    #         verbose=True
    #     )
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # Important!
    #         },
        # }


## CNNLSTM scalie

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components, mu_scale=10, mu_bias=.9, sigma_scale=10.0):
    """
    Split raw MDN parameters into (pi, mu, sigma).

    Args:
        raw_params: (B, 3 * K) from the network
        n_components: number of mixture components
        mu_scale: scaling factor for mu (default 1.0 = no scaling)
        mu_bias: shift/bias applied after scaling
        sigma_scale: scaling factor for sigma (default 10.0)
    """
    B = raw_params.size(0)
    raw = raw_params.view(B, n_components, 3)

    pi_raw = raw[..., 0]
    mu_raw = raw[..., 1]
    sigma_raw = raw[..., 2]

    pi = F.softmax(pi_raw, dim=-1)
    mu = mu_raw / mu_scale + mu_bias
    sigma = F.softplus(sigma_raw / sigma_scale) + 1e-4

    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] # REMOVED redundant transposes

        # --- Debug print first candle ---
        # if x.ndim == 3:  # batched: (B, T, F)
        #     first_candle = x[0, 0, :]   # first sample, first time step, all features
        #     print("First candle features:", first_candle.detach().cpu().numpy())
        # elif x.ndim == 2:  # single sequence: (T, F)
        #     first_candle = x[0, :]      # first time step, all features
        #     print("First candle features:", first_candle.detach().cpu().numpy())
        # else:
        #     print("Unexpected shape for x:", x.shape)
        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU

        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    # # Inside your CNNLSTM_MDN class
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=1e-4)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
    #     # Define the scheduler
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode='min',      # We want to minimize the validation loss
    #         factor=0.5,      # Reduce LR by 80% (1.0 -> 0.2)
    #         patience=10,      # Wait 5 validation epochs with no improvement before reducing
    #         verbose=True
    #     )
        
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # The metric to watch
    #         },
    #     }
    # def configure_optimizers(self):
    #     return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


# train

In [13]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.make_step import make_step
from utils.padding_batch_reg import collate_batch
from utils.get_init_argumens import get_init_args
import pandas as pd
import io
import numpy as np
import os
from add_ons.drop_column import drop_columns
from add_ons.normalize_candle_seq import add_label_normalized_candles
from add_ons.feature_pipeline3 import FeaturePipeline
from add_ons.candle_dif_seq import add_candle_differences
from sklearn.metrics import accuracy_score, f1_score
# ---------------- Evaluation ---------------- #
@torch.no_grad()
def evaluate_model_mdn(model, val_loader, zero_idx=0, threshold=0.1):
    """
    Evaluate CNN–LSTM–MDN model (last-output version).

    Args
    ----
    model : pl.LightningModule with MDN forward
    val_loader : DataLoader yielding (X, y, lengths)
    zero_idx : which mixture component is considered "no-line" (usually 0)
    threshold : if pi[:,zero_idx] > threshold → predict invalid

    Returns
    -------
    dict with mse, mae, acc, f1
    """
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    device = next(model.parameters()).device

    with torch.no_grad():
        for X_batch, y_batch, lengths in val_loader:
            if isinstance(X_batch, dict):
                X_batch = {k: v.to(device) for k, v in X_batch.items()}
            else:
                X_batch = X_batch.to(device)

            y_batch = y_batch.to(device)
            mdn = model(X_batch, lengths)
            pi, mu, sigma = mdn["pi"], mdn["mu"], mdn["sigma"]  # (B,K)

            # regression expectation
            y_pred = (pi * mu).sum(dim=-1)  # (B,)
            B = y_batch.size(0)
            y_len = (y_batch > 0).sum(dim=1)                # (B,)
            idx = torch.clamp(y_len - 1, min=0)             # last valid index
            y_true = y_batch[torch.arange(B, device=y_batch.device), idx]  # (B,)
            # only last step
            # print("lengths(features):", lengths[:10])
            # print("lengths(labels):", y_len[:10])

            all_preds_reg.append(y_pred.cpu().numpy())
            all_labels_reg.append(y_true.cpu().numpy())

            # validity classification
            pi_zero = pi[:, zero_idx]  # (B,)
            pred_valid = (pi_zero < (1 - threshold)).long()
            true_valid = torch.ones_like(pred_valid)  # last step always valid

            all_preds_len.extend(pred_valid.cpu().numpy().tolist())
            all_labels_len.extend(true_valid.cpu().numpy().tolist())


        # ----- Regression metrics -----
    all_preds_reg = np.concatenate(all_preds_reg)  # (N,)
    all_labels_reg = np.concatenate(all_labels_reg)
    mse = ((all_preds_reg - all_labels_reg) ** 2).mean()
    mae = np.abs(all_preds_reg - all_labels_reg).mean()
    # ----- Validity metrics -----
    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len, average="macro")

    print("\n📊 Validation Metrics (MDN, last-output):")
    print(f"  Regression → MSE: {mse:.6f}, MAE: {mae:.6f}")
    print(f"  Validity   → Acc: {acc:.4f}, F1: {f1:.4f}")

    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=200,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=1000,
    save_model=False,
    return_val_accuracy = True,
    test_mode = False
):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_label_normalized_candles),
            make_step(add_candle_differences),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
            
        ],
        norm_methods={
            "main": {
                "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
                "upper_body_ratio": "standard", "lower_body_ratio": "standard",
                "upper_lower_body_ratio": "standard", "Candle_Color": "standard"
            }
        },
        per_window_flags=[True, True, True]
    )
    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True,
            feature_pipeline=pipeline
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False,
            feature_pipeline=pipeline
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = CNNLSTM_MDN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        lr=lr
    )
    init_args = get_init_args(model, input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, lr=lr)

    model_class_info = {
        "module": model.__class__.__module__,
        "class": model.__class__.__name__,
        "init_args": init_args
    }

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    
    # --- Debug / Test mode --- #
    if test_mode:
        save_model = False
        from itertools import islice

        # Try to grab 3rd batch; if not available, take first
        try:
            batch = next(islice(iter(train_loader), 2, 3))
        except StopIteration:
            batch = next(iter(train_loader))

        X_batch_dict, y_batch, lengths = batch

        print("🔍 Debug batch:")
        if isinstance(X_batch_dict, dict):
            print("  Keys in X_batch:", list(X_batch_dict.keys()))
        print("  y_batch shape:", y_batch.shape)
        print("  First label in batch:", y_batch[0])

        # --- Track real column names for each feature group ---
        feature_names_dict = {}
        for name, X_batch in X_batch_dict.items():
            if name == "main":
                # Use actual feature columns after preprocessing
                feature_names_dict[name] = feature_cols
            else:
                # For extra feature groups, fallback to generic names
                feature_names_dict[name] = [f"{name}_{i}" for i in range(X_batch.shape[2])]

        dfs = []
        for name, X_batch in X_batch_dict.items():
            print(f"\nFeature group: {name}")
            print("  X_batch shape:", X_batch.shape)
            print("  First sequence in batch (first  steps):\n", X_batch[0][:])

            batch_size_, seq_len, feature_dim = X_batch.shape
            df_part = pd.DataFrame(
                X_batch.reshape(batch_size_ * seq_len, feature_dim).numpy(),
                columns=feature_names_dict[name]
            )
            dfs.append(df_part)

        # Combine all feature groups horizontally
        global df_seq
        df_seq = pd.concat(dfs, axis=1)
        print("\n✅ Combined df_seq shape:", df_seq.shape)
        print("✅ Column names in df_seq:", df_seq.columns.tolist())


    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm"
    )
    trainer.fit(model, train_loader, val_loader)

    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,
    "max_len_y": max_len_y,
    "feature_cols": feature_cols,
    "scalers": pipeline.scalers,
    "pipeline_config": pipeline.export_config(),
    "model_class_info": model_class_info   # ✅ save model class info
}, meta_out)
        
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model_mdn(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv",
        "/home/iatell/projects/meta-learning/data/seq_line_labels.csv",
        save_model=True,
        do_validation=True,
        test_mode = True
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name     | Type        | Params | Mode 
--------------------------------------------------
0  | fc1      | Linear      | 1.0 K  | train
1  | ln1      | LayerNorm   | 128    | train
2  | fc2      | Linear      | 2.1 K  | train
3  | ln2      | LayerNorm   | 64     | train
4  | conv1    | Conv1d      | 2.1 K  | train
5  | bn1      | BatchNorm1d | 128    | train
6  | conv3    | Conv1d      | 6.2 K  | train
7  | bn3      | BatchNorm1d | 128    | train
8  | mixer    | Conv2d      | 3      | train
9  | lstm     |

🔍 Debug batch:
  Keys in X_batch: ['main']
  y_batch shape: torch.Size([32, 6])
  First label in batch: tensor([1.0157, 1.0583, 0.0000, 0.0000, 0.0000, 0.0000])

Feature group: main
  X_batch shape: torch.Size([32, 56, 15])
  First sequence in batch (first  steps):
 tensor([[ 1.3218,  0.0086, -0.8959,  0.9549,  2.4932, -0.3550,  2.5471,  1.1624,
          1.2290,  1.1575,  1.1680,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4148, -0.8214, -0.4183, -1.0472, -0.6969, -0.7031, -0.5979,  1.1674,
          1.1787,  1.0636,  1.0884,  0.0043, -0.0409, -0.0811, -0.0682],
        [ 1.0012, -0.6900, -0.6813, -1.0472, -0.2742, -0.7686,  1.6776,  1.0885,
          1.1401,  1.0093,  1.0230, -0.0675, -0.0328, -0.0511, -0.0600],
        [ 0.3101,  0.2795, -0.3933,  0.9549, -0.1585, -0.4093, -0.0576,  1.0235,
          1.0881,  0.9979,  1.0564, -0.0597, -0.0456, -0.0112,  0.0326],
        [ 0.5436, -0.4985, -0.1458, -1.0472, -0.2407, -0.4049, -0.1756,  1.0564,
          1.0946,  0.9752,  1.0109, 

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

First candle features: [ 2.4509761   0.06757022 -0.425552    0.9548821   2.4932256   0.43385926
  1.712948    1.6238451   1.6838083   1.609801    1.6305033   0.
  0.          0.          0.        ]


Validation: |          | 0/? [00:00<?, ?it/s]

First candle features: [ 0.34788373  0.2923229  -0.8116072   0.9548821  -0.16202562 -0.7484566
  1.825419    1.502845    1.6000752   1.4908248   1.5525004   0.
  0.          0.          0.        ]


`Trainer.fit` stopped: `max_steps=1` reached.


First candle features: [ 0.34788373  0.2923229  -0.8116072   0.9548821  -0.16202562 -0.7484566
  1.825419    1.502845    1.6000752   1.4908248   1.5525004   0.
  0.          0.          0.        ]

📊 Validation Metrics (MDN, last-output):
  Regression → MSE: 0.060597, MAE: 0.191716
  Validity   → Acc: 1.0000, F1: 1.0000


In [15]:
df_seq = df_seq.loc[~(df_seq==0).all(axis=1)]
df_seq

,upper_shadow,body,lower_shadow,Candle_Color,upper_body_ratio,lower_body_ratio,upper_lower_body_ratio,open_prop,high_prop,low_prop,close_prop,open_dif,high_dif,low_dif,close_dif
0,1.321786,0.008634,-0.895926,0.954882,2.493226,-0.355042,2.547058,1.162368,1.228990,1.157517,1.167987,0.000000,0.000000,0.000000,0.000000
1,-0.414832,-0.821408,-0.418283,-1.047250,-0.696935,-0.703118,-0.597881,1.167369,1.178731,1.063608,1.088384,0.004302,-0.040894,-0.081130,-0.068154
2,1.001162,-0.689979,-0.681345,-1.047250,-0.274219,-0.768598,1.677601,1.088532,1.140088,1.009280,1.023049,-0.067534,-0.032784,-0.051078,-0.060029
3,0.310094,0.279493,-0.393333,0.954882,-0.158510,-0.409299,-0.057559,1.023519,1.088113,0.997934,1.056417,-0.059725,-0.045589,-0.011242,0.032616
4,0.543558,-0.498462,-0.145810,-1.047250,-0.240747,-0.404925,-0.175599,1.056417,1.094575,0.975182,1.010898,0.032142,0.005939,-0.022799,-0.043088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,-0.059996,-1.183145,-0.249545,-1.047250,-0.669764,-0.727806,-0.448530,1.117452,1.139657,0.964626,0.997559,0.002254,-0.014801,-0.104581,-0.107104
1746,-0.024688,0.020147,0.508013,0.954882,1.388192,2.264179,-0.670333,0.999270,1.029305,0.934067,1.006226,-0.105760,-0.096829,-0.031680,0.008688
1747,0.666663,-0.023924,-0.033323,0.954882,2.493226,2.264179,-0.201301,1.006226,1.051484,0.964592,1.008548,0.006961,0.021547,0.032680,0.002308
1748,-0.437450,-0.562835,-0.713843,-1.047250,-0.657662,-0.752152,-0.318531,1.008548,1.019369,0.942857,0.955410,0.002308,-0.030542,-0.022533,-0.052688


# server

In [11]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import glob
import joblib
import torch
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
from servers.pre_process.multi_reg_dif_seq import ServerPreprocess, import_class, build_pipeline_from_config
# from models.LSTM.cnn_lstm_mdn import CNNLSTM_MDN  # <-- your updated "last-output" model

app = Flask(__name__)

# ---------------- Load model and meta ----------------
meta_path = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_meta_multireg_*.pkl")[0]
state_path = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_model_multireg*.pt")[0]

meta = joblib.load(meta_path)
FEATURES = meta['feature_cols']
print("features",FEATURES)
# ---------------- Model ----------------
# Reconstruct model class
#for python file:
# model_cls_info = meta["model_class_info"]
# ModelClass = import_class(model_cls_info["module"], model_cls_info["class"])
model_cls_info = meta["model_class_info"]
ModelClass = CNNLSTM_MDN
# Initialize model with original args
model = ModelClass(**model_cls_info["init_args"])
model = CNNLSTM_MDN.load_from_checkpoint(state_path)
model.eval()

# ---------------- Load data ----------------
df = pd.read_csv( "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv", parse_dates=['timestamp'])

# ---------------- Setup pipeline ----------------
pipeline = build_pipeline_from_config(meta["pipeline_config"])
pipeline.scalers = meta["scalers"]

# Stateful preprocessing instance
preproc = ServerPreprocess(feature_pipeline=pipeline)


# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("sequential.html")


@app.route("/get_and_add_data")
def get_and_add_data():
    dense = df.set_index('timestamp').asfreq('D').ffill()
    initial_seq_len = 21
    next_idx = request.args.get("idx", type=int)
    if next_idx is None:
        # First call → load initial candles
        if len(preproc.dataset) == 0:
            for _, row in dense.iloc[:initial_seq_len].iterrows():
                preproc.add_candle(row)
        candles = [
            {'time': int(ts.timestamp()),
             'open': float(row.open),
             'high': float(row.high),
             'low': float(row.low),
             'close': float(row.close)}
            for ts, row in dense.iloc[:initial_seq_len].iterrows()
        ]
        print("Returning initial candles:", candles)

        return jsonify({
            "initial_seq_len": initial_seq_len,
            "next_idx": initial_seq_len,
            "candles": candles
        })
    else:
        # Subsequent calls → 1 candle
        if next_idx >= len(dense):
            print("Reached end of data at index:", next_idx)
            return jsonify({"error": "End of data"}), 404

        row = dense.iloc[next_idx]
        candle = {
            'time': int(row.name.timestamp()),
            'open': float(row.open),
            'high': float(row.high),
            'low': float(row.low),
            'close': float(row.close)
        }

        # ✅ Add to preproc automatically
        preproc.add_candle(row)

        return jsonify({
            "next_idx": next_idx + 1,
            "candle": candle
        })


@app.route("/predict", methods=['POST'])
def predict():
    data = request.get_json(force=True)
    seq_len = data.get("seq_len")

    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Provide 'seq_len' as an int"}), 400

    try:
        # prepare subsequence from current state
        seq_df = preproc.prepare_seq(seq_len)
    except ValueError as e:
        return jsonify({"error": str(e)}), 400

    X_np = seq_df[FEATURES].values.astype(np.float32)
    X_t = torch.from_numpy(X_np).unsqueeze(0)
    print("XT",X_t)
    dict_x = {"main": X_t}
    with torch.no_grad():
        mdn_out = model(dict_x)

    pi    = mdn_out['pi'][0].cpu().numpy()
    mu    = mdn_out['mu'][0].cpu().numpy()
    sigma = mdn_out['sigma'][0].cpu().numpy()
    last_close =preproc.dataset.iloc[-1]['close']
    return jsonify({
        'pred_prices': (last_close * mu).tolist(),
        'pred_sigmas': (last_close * sigma).tolist(),
        'pi': pi.tolist()
    })

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


features ['upper_shadow', 'body', 'lower_shadow', 'Candle_Color', 'upper_body_ratio', 'lower_body_ratio', 'upper_lower_body_ratio', 'open_prop', 'high_prop', 'low_prop', 'close_prop']
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Sep/2025 00:59:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:35] "GET /get_and_add_data?init=1 HTTP/1.1" 200 -


Returning initial candles: [{'time': 1514764800, 'open': 13707.91, 'high': 13818.55, 'low': 12750.0, 'close': 13380.0}, {'time': 1514851200, 'open': 13382.16, 'high': 15473.49, 'low': 12890.02, 'close': 14675.11}, {'time': 1514937600, 'open': 14690.0, 'high': 15307.56, 'low': 14150.0, 'close': 14919.51}, {'time': 1515024000, 'open': 14919.51, 'high': 15280.0, 'low': 13918.04, 'close': 15059.54}, {'time': 1515110400, 'open': 15059.56, 'high': 17176.24, 'low': 14600.0, 'close': 16960.39}, {'time': 1515196800, 'open': 16960.39, 'high': 17143.13, 'low': 16011.21, 'close': 17069.79}, {'time': 1515283200, 'open': 17069.79, 'high': 17099.96, 'low': 15610.0, 'close': 16150.03}, {'time': 1515369600, 'open': 16218.85, 'high': 16322.3, 'low': 12812.0, 'close': 14902.54}, {'time': 1515456000, 'open': 14902.54, 'high': 15500.0, 'low': 14011.05, 'close': 14400.0}, {'time': 1515542400, 'open': 14401.0, 'high': 14955.66, 'low': 13131.31, 'close': 14907.09}, {'time': 1515628800, 'open': 14940.0, 'high'

127.0.0.1 - - [08/Sep/2025 00:59:36] "GET /get_and_add_data?idx=21 HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 00:59:36] "POST /predict HTTP/1.1" 200 -


XT XT XT XT XT tensor([[[ 0.1086,  1.0951, -0.8802,  0.9549, -0.6432, -0.8718,  2.3966,
           1.0664,  1.2174,  1.0606,  1.1896],
         [-0.8142, -1.1374,  0.1731, -1.0472, -0.7914, -0.6200, -0.9155,
           1.1896,  1.1896,  1.0190,  1.0716],
         [ 0.3998, -0.7047,  0.7826, -1.0472, -0.4533, -0.1947, -0.5962,
           1.0716,  1.1084,  0.9201,  1.0000]]])
First candle features: [ 0.10861079  1.0951006  -0.88018584  0.9548821  -0.6431555  -0.87184983
  2.3965523   1.0664431   1.2173735   1.0606318   1.1895801 ]
tensor([[[ 1.9777, -0.0554,  0.0937, -1.0472,  2.4932,  2.2642,  0.2762,
           1.0198,  1.1040,  0.9698,  1.0188],
         [ 0.1740,  0.3977,  0.2507,  0.9549, -0.3832, -0.1640, -0.5461,
           1.0186,  1.0962,  0.9628,  1.0664],
         [ 0.1086,  1.0951, -0.8802,  0.9549, -0.6432, -0.8718,  2.3966,
           1.0664,  1.2174,  1.0606,  1.1896],
         [-0.8142, -1.1374,  0.1731, -1.0472, -0.7914, -0.6200, -0.9155,
           1.1896,  1.1896,  1.0